# Teoria da Computação

### Quarta lista de exercícios
#### Máquina de Mealy e Máquina de Moore
#### Prof. Dr. Márcio Basgalupp



Aron Ifanger Maciel, 17 de abril de 2018

### Exercício 1

*Defina formalmente a função programa estendida para as Máquinas de Mealy e de Moore.*

#### Resposta:

A **Máquina de Mealy** é definida como uma 6-upla:

<center> $ M = (\Sigma, Q, \delta, q_0, F, \Delta) $ </center>

onde:

* $\Sigma$: é um alfabeto de síbolos de entrada;
* $Q$: conjunto de estados possíveis;
* $\delta$: função programa, **definida como: 
<br> <center> $ \delta: Q \times \Sigma \rightarrow Q \times \Delta^* $** </center>
* $q_0$: estado inicial;
* $F$: conjunto de estados finais;
* $\Delta$: alfabeto de símbolos de saída.

A **Máquina de Moore** é definida como uma 7-upla:

<center> $ M = (\Sigma, Q, \delta, q_0, F, \Delta, \delta_S) $ </center>

onde:

* $\Sigma$: é um alfabeto de síbolos de entrada;
* $Q$: conjunto de estados possíveis;
* $\delta$: função programa, **definida como: 
<br> <center> $ \delta: Q \times \Sigma \rightarrow Q $** </center>
* $q_0$: estado inicial;
* $F$: conjunto de estados finais;
* $\Delta$: alfabeto de símbolos de saída.
* $\delta_S$: função de saída, **definida como: 
<br> <center>$ \delta_S: Q \rightarrow \Delta^* $ **</center>


### Exercício 2

*Desenvolva uma Máquina de Mealy e uma de Moore que realizem a conversão da repreentação de valores monetários de dólares para reais. Por exemplo, dado o valor US\$25,010.59 deve ser gravado o valor R\$25.010,59 na fita de saída (atenção para o uso da vírgula e do ponto nos dois valores). Adicionalmente, os autômatos deve verificar se a entrada é um valor monetário válido.*

### Exercício 3

*Considere o exemplo de diálogo apresentado para a Máquina de Mealy:*
    
*a. Desenvolva uma Máquina de Moore que realize o mesmo processamento*

### Exercício 4

*Desenvolva um programa de computador que simule o processamento de qualquer Autômato Finito Determinístico, como segue:*

*a. Entrada: função de transição $ \delta $, estado inicial, conjunto de estados finais e as palavras a serem processadas;*
   
*b. Saída: condição de parada ACEITA/REJEITA e identificação do estado de parada.*
    


In [3]:
# Implementação de uma classe estado

class Estado(object):
    
    def __init__(self, nome, inicial = False, final = False):
        self.nome = nome
        self.transicao = dict()
        self.inicial = inicial
        self.final = final
    
    def criaTransicao(self, palavra, estado):
        print("Criando transicao para o automato %s: {%s: %s}"%(self.nome, palavra, estado.nome))
        self.transicao.update({palavra:estado})
        
    def processa(self, palavra):
        return(self.transicao[palavra])
    
    def imprime(self):
        tipo = {(True, True): "inicial e final", (True, False): "inicial", (False, True): "final", (False, False): ""}
        print("Estado %s %s. Mapa: %s"%(self.nome, tipo[(self.inicial, self.final)], self.transicao))

In [211]:
# Exemplo de uso da classe estado

q1 = Estado("q1")
q2 = Estado("q2")

q1.criaTransicao("a", q2)
q2.criaTransicao("b", q1)

q1.processa("a").processa("b").nome

Criando transicao para o automato q1: {a: q2}
Criando transicao para o automato q2: {b: q1}


'q1'

In [230]:
# Implementação da classe Automato

class Automato(object):
    
    def __init__(self, func_transicao, estado_inicial, estados_finais):
        self.func_transicao = func_transicao
        self.estado_inicial = estado_inicial
        self.estados_finais = estados_finais
        
        self.nome_estados = list(set([estado_inicial] + [t for trans in list(func_transicao.values()) for t in list(trans.values())]))
        self.estados = [Estado(n, n == estado_inicial, n in estados_finais) for n in self.nome_estados]
        self.mapa_estados = dict(zip(self.nome_estados, self.estados))
        
        for estado in func_transicao:
            for transicao in func_transicao[estado]:
                self.mapa_estados[estado].criaTransicao(transicao, self.mapa_estados[func_transicao[estado][transicao]])
        
        self.estado_atual = mapa_estados[estado_inicial]
        
    def criaTransicoes(self, estado, dict_trans):
        [estado.criaTransicao(t, dict_trans[t]) for t in dict_trans]
    
    def processa(self, palavra):
        self.estado_atual = self.estado_atual.processa(palavra)

In [231]:
estado_inicial = "q0"
estados_finais = ["q1", "q2"]
func_transicao = {
    "q0":{"a": "q1"},
    "q1":{"a": "q2", "b": "q2"},
    "q2":{"b": "q1"}
}
palavras = ["a", "b", "b"]

In [1]:
def processa_palavra(func_transicao, estado_inicial, estados_finais, palavras):
    automato = Automato(func_transicao, estado_inicial, estados_finais)
    
    for palavra in palavras:
        automato.processa(palavra)

    return("ACEITA" if automato.estado_atual.final else "REJEITA")

In [2]:
processa_palavra(func_transicao, estado_inicial, estados_finais, palavras)

NameError: name 'func_transicao' is not defined

## Implementação da função que processa um AFD

In [72]:
def afd(funcaoTransicao, estadoInicial, estadosFinais, palavra):
    estadoAtual = estadoInicial
    
    for letra in palavra:
        try: estadoAtual = funcaoTransicao[estadoAtual][letra]
        except: return("REJEITA", estadoAtual, letra)
    
    return("ACEITA" if estadoAtual in estadosFinais else "REJEITA")

## Exemplo de uso para reconhecimento de palavras com número par de 'a' e de 'b'

In [73]:
funcaoTransicao = {
    "q0": {"a": "q1", "b": "q2"},
    "q1": {"a": "q0", "b": "q3"},
    "q2": {"a": "q3", "b": "q0"},
    "q3": {"a": "q2", "b": "q1"},
}
estadoInicial = "q0"
estadosFinais = ["q0"]
palavra = "abba"

print(afd(funcaoTransicao, estadoInicial, estadosFinais, palavra))

ACEITA


## Implementação da função que simula a Máquina de Moore

In [179]:
def moore(funcaoTransicao, estadoInicial, estadosFinais, funcaoSaida, palavra):
    estadoAtual = estadoInicial
    saida = ""
    for letra in palavra:
        try: estadoAtual = funcaoTransicao[estadoAtual][letra]
        except: return("REJEITA: %s"% palavra)
        try: saida += str(funcaoSaida[estadoAtual])
        except: None
    
    return("ACEITA: %s -> %s"%(palavra, saida) if estadoAtual in estadosFinais else "REJEITA")

## Exemplo de uso da Máquina de Moore
Uma vez que a Máquina de Moore faz as impressões nos estados, é necessário que exista um estado para cada possível impressão do alfabeto, gerando a necessidade de gerar muitos estados para produzir o resultado esperado. Em função disto foram implementadas algumas funções que auxiliam a geração das funções de transição para cada um destes estados.

In [182]:
# Gera um estado para cada elemento da lista de caractere 
# Exemplo: geraDictMoore("q3", range(10),"q4") gera o seguinte dicionário:
# {'q3': {'0': 'q4-0', '1': 'q4-1', '2': 'q4-2', '3': 'q4-3', '4': 'q4-4', 
#         '5': 'q4-5', '6': 'q4-6', '7': 'q4-7', '8': 'q4-8', '9': 'q4-9'}}
def geraDictMoore(chave, listaCaractere, valor, adicional = []): 
    return({chave:dict([(str(i),valor+"-"+str(i)) for i in listaCaractere] + adicional)})

# Gera os caracteres de saída
# Exemplo: geraSaidaMoore("q4") gera o seguinte dicionário
# {'q4-0': 0,'q4-1': 1,'q4-2': 2,'q4-3': 3,'q4-4': 4,'q4-5': 5,'q4-6': 6,'q4-7': 7,'q4-8': 8,'q4-9': 9}
def geraSaidaMoore(chave, lista = range(10)): 
    return(dict([("%s-%d"%(chave,i), i) for i in lista]))

# Criação do dicinário que representa as funções de transição para a conversão da representação dos valores monetários 
# (Exercício 2)
ft = {}
ft.update({"q0":{"U":"q1"}})
ft.update({"q1":{"S":"q2"}})
ft.update({"q2":{"$":"q3"}})
ft.update(geraDictMoore("q3", range(10),"q4"))
for estado in ft["q3"].values(): ft.update(geraDictMoore(estado, range(10),"q5",[(",","q7"),(".","q10")]))
for estado in ft["q4-1"].values(): ft.update(geraDictMoore(estado, range(10),"q6",[(",","q7"),(".","q10")]))
for estado in ft["q5-1"].values(): ft.update(geraDictMoore(estado, [],"",[(",","q7"),(".","q10")]))
ft.update(geraDictMoore("q7", range(10),"q8"))
for estado in ft["q7"].values(): ft.update(geraDictMoore(estado, range(10), "q9"))
for estado in ft["q8-1"].values(): ft.update(geraDictMoore(estado, range(10), "q6"))
ft.update(geraDictMoore("q10", range(10),"q11"))
for estado in ft["q10"].values(): ft.update(geraDictMoore(estado, range(10), "q12"))

# Criação do dicinário que gera a saída para o exercício de conversão da representação dos valores monetários 
# Este dicionário atribui a cada estado, um possível caractere de impressão. 
fs = {}
fs.update({"q2":"R"})
fs.update({"q3":"$"})
fs.update(geraSaidaMoore("q4"))
fs.update(geraSaidaMoore("q5"))
fs.update(geraSaidaMoore("q6"))
fs.update({"q7":".", "q10":","})
fs.update(geraSaidaMoore("q8"))
fs.update(geraSaidaMoore("q9"))
fs.update(geraSaidaMoore("q11"))
fs.update(geraSaidaMoore("q12"))

estadoInicial = "q0"
estadosFinais = geraDictMoore("q11", range(10), "q12")["q11"].values()

# Exemplos de casos que são aceitos pela Márquina de Moore deste exercício
print(moore(ft, estadoInicial, estadosFinais, fs, palavra = "US$123,100.09"))
print(moore(ft, estadoInicial, estadosFinais, fs, palavra = "US$6,254.18"))
print(moore(ft, estadoInicial, estadosFinais, fs, palavra = "US$23.27"))
print(moore(ft, estadoInicial, estadosFinais, fs, palavra = "US$8.53"))
print(moore(ft, estadoInicial, estadosFinais, fs, palavra = "US$0.53"))

# Exemplos de casos que são aceitos pela Márquina de Moore deste exercício
print(moore(ft, estadoInicial, estadosFinais, fs, palavra = "US$,100.09"))
print(moore(ft, estadoInicial, estadosFinais, fs, palavra = "U$6,254.18"))
print(moore(ft, estadoInicial, estadosFinais, fs, palavra = "S$23.27"))
print(moore(ft, estadoInicial, estadosFinais, fs, palavra = "$8.53"))
print(moore(ft, estadoInicial, estadosFinais, fs, palavra = "R$0.53"))
print(moore(ft, estadoInicial, estadosFinais, fs, palavra = "R$0.541"))

ACEITA: US$123,100.09 -> R$123.100,09
ACEITA: US$6,254.18 -> R$6.254,18
ACEITA: US$23.27 -> R$23,27
ACEITA: US$8.53 -> R$8,53
ACEITA: US$0.53 -> R$0,53
REJEITA: US$,100.09
REJEITA: U$6,254.18
REJEITA: S$23.27
REJEITA: $8.53
REJEITA: R$0.53
REJEITA: R$0.541


## Implementação da função que simula a Máquina de Mealy

In [203]:
def moore(funcaoTransicao, estadoInicial, estadosFinais, funcaoSaida, palavra):
    estadoAtual = estadoInicial
    saida = ""
    for letra in palavra:
        try: saida += str(funcaoSaida[(estadoAtual, letra)])
        except: None
        try: estadoAtual = funcaoTransicao[estadoAtual][letra]
        except: return("REJEITA: %s"% palavra)
    
    return("ACEITA: %s -> %s"%(palavra, saida) if estadoAtual in estadosFinais else "REJEITA")

## Exemplo de uso da Máquina de Mealy
Para facilitar a geração das funções de transição, também foram implementadas funções auxiliares.

In [216]:
def geraDictMealy(chave, listaLetras = [], valor = "", adicional = []):
    return({chave: dict([(str(letra), valor) for letra in listaLetras] + adicional)})

def geraSaidaMealy(chave, listaLetra = range(10)):
    return(dict([((chave, str(letra)), str(letra)) for letra in listaLetra]))

# Criação do dicinário que representa as funções de transição para a conversão da representação dos valores monetários 
# (Exercício 2)
ft = {}
ft.update({"q0":{"U":"q1"}})
ft.update({"q1":{"S":"q2"}})
ft.update({"q2":{"$":"q3"}})
ft.update(geraDictMealy("q3", range(10), "q4"))
ft.update(geraDictMealy("q4", range(10), "q5", [(",","q7"),(".","q10")]))
ft.update(geraDictMealy("q5", range(10), "q6", [(",","q7"),(".","q10")]))
ft.update(geraDictMealy("q6", adicional = [(",","q7"),(".","q10")]))
ft.update(geraDictMealy("q7", range(10),"q8"))
ft.update(geraDictMealy("q8", range(10), "q9"))
ft.update(geraDictMealy("q9", range(10), "q6"))
ft.update(geraDictMealy("q10", range(10), "q11"))
ft.update(geraDictMealy("q11", range(10), "q12"))

# Criação do dicinário que gera a saída para o exercício de conversão da representação dos valores monetários 
# Este dicionário atribui a cada estado, um possível caractere de impressão. 
fs = {}
fs.update({("q1","S"):"R"})
fs.update({("q2","$"):"$"})
fs.update(geraSaidaMealy("q3"))
fs.update({("q3", ","):".", ("q3", "."):","})
fs.update(geraSaidaMealy("q4"))
fs.update({("q4", ","):".", ("q4", "."):","})
fs.update(geraSaidaMealy("q5"))
fs.update({("q5", ","):".", ("q5", "."):","})
fs.update({("q6",","):".", ("q6","."):","})
fs.update(geraSaidaMealy("q7"))
fs.update(geraSaidaMealy("q8"))
fs.update(geraSaidaMealy("q9"))
fs.update(geraSaidaMealy("q10"))
fs.update({("q10",","):".", ("q10","."):","})
fs.update(geraSaidaMealy("q11"))

estadoInicial = "q0"
estadosFinais = geraDictMealy("q11", range(10), "q12")["q11"].values()

# Exemplos de casos que são aceitos pela Márquina de Moore deste exercício
print(moore(ft, estadoInicial, estadosFinais, fs, palavra = "US$123,100.09"))
print(moore(ft, estadoInicial, estadosFinais, fs, palavra = "US$6,254.18"))
print(moore(ft, estadoInicial, estadosFinais, fs, palavra = "US$23.27"))
print(moore(ft, estadoInicial, estadosFinais, fs, palavra = "US$8.53"))
print(moore(ft, estadoInicial, estadosFinais, fs, palavra = "US$0.53"))

# Exemplos de casos que são aceitos pela Márquina de Moore deste exercício
print(moore(ft, estadoInicial, estadosFinais, fs, palavra = "US$,100.09"))
print(moore(ft, estadoInicial, estadosFinais, fs, palavra = "U$6,254.18"))
print(moore(ft, estadoInicial, estadosFinais, fs, palavra = "S$23.27"))
print(moore(ft, estadoInicial, estadosFinais, fs, palavra = "$8.53"))
print(moore(ft, estadoInicial, estadosFinais, fs, palavra = "R$0.53"))
print(moore(ft, estadoInicial, estadosFinais, fs, palavra = "R$0.541"))

ACEITA: US$123,100.09 -> R$123.100,09
ACEITA: US$6,254.18 -> R$6.254,18
ACEITA: US$23.27 -> R$23,27
ACEITA: US$8.53 -> R$8,53
ACEITA: US$0.53 -> R$0,53
REJEITA: US$,100.09
REJEITA: U$6,254.18
REJEITA: S$23.27
REJEITA: $8.53
REJEITA: R$0.53
REJEITA: R$0.541
